In [1]:
import pandas as pd
from sklearn.preprocessing import StandardScaler, LabelEncoder
from imblearn.under_sampling import RandomUnderSampler

# 1. 데이터 로드
file_path = "train.csv"
df = pd.read_csv(file_path)

# 2. 결측치 처리 (평균/최빈값 대체 또는 삭제)
df.fillna(df.median(numeric_only=True), inplace=True)  # 숫자형 컬럼은 중앙값으로 대체
df.fillna(df.mode().iloc[0], inplace=True)  # 범주형 컬럼은 최빈값으로 대체

# 3. 데이터 타입 변환 (범주형 컬럼 Label Encoding)
categorical_columns = df.select_dtypes(include=['object']).columns
label_encoders = {}
for col in categorical_columns:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])
    label_encoders[col] = le  # 나중에 역변환을 위해 저장

# 4. 데이터 불균형 처리 (언더샘플링)
if "임신 성공 여부" in df.columns:
    X = df.drop(columns=["임신 성공 여부"])
    y = df["임신 성공 여부"]
    undersample = RandomUnderSampler(sampling_strategy=0.5, random_state=42)
    X_resampled, y_resampled = undersample.fit_resample(X, y)
    df_balanced = pd.concat([X_resampled, y_resampled], axis=1)
else:
    df_balanced = df

# 5. 데이터 정규화 (StandardScaler 적용)
scaler = StandardScaler()
numerical_columns = df_balanced.select_dtypes(include=['int64', 'float64']).columns
df_balanced[numerical_columns] = scaler.fit_transform(df_balanced[numerical_columns])

# 6. 전처리된 데이터 저장
preprocessed_csv_path = "preprocessed_train.csv"
df_balanced.to_csv(preprocessed_csv_path, index=False, encoding="utf-8-sig")
